In [ ]:
!pip install pylibtiff

In [13]:
import cv2
from PIL import Image
import os
import glob
import matplotlib.pyplot as plt
import numpy as np
from PIL import ExifTags
import imageio as io
import pandas as pd
import yaml
import sys
from matplotlib import patches
from pycocotools import mask as M
import tifffile
# from libtiff import TIFF

sys.path.append('../scripts')
import utils

In [6]:
img1 = tifffile.imread('../tiff_thermal.tiff')
img2 = np.array(Image.open('../thermal2.jpg'))

In [31]:
# bw_thermal = Image.open('../../../dataset/bw_thermal.tiff')

In [23]:
thermal_images = glob.glob('../../../dataset/attachments/*.tiff')
thermal_images

['../../../dataset/attachments\\img_thermo_1710240449484.tiff',
 '../../../dataset/attachments\\img_thermo_1710240454797.tiff']

In [41]:
with Image.open('../../../dataset/bw_thermal.tiff', 'r') as f:

uint8


In [24]:
tiff_img = Image.open(thermal_images[0], 'r')

In [29]:
tiff_img.getchannel('A')

array([[255, 255, 255, ..., 255, 255, 255],
       [255, 255, 255, ..., 255, 255, 255],
       [255, 255, 255, ..., 255, 255, 255],
       ...,
       [255, 255, 255, ..., 255, 255, 255],
       [255, 255, 255, ..., 255, 255, 255],
       [255, 255, 255, ..., 255, 255, 255]], dtype=uint8)

In [36]:
for i in list(bw_thermal.tag):
    print(i, ExifTags.TAGS[i],tiff_img.tag[i])
# min_max_norm = lambda x: (x - np.min(x)) / (np.max(x) - np.min(x))

256 ImageWidth (320,)
257 ImageLength (240,)
258 BitsPerSample (8, 8, 8, 8)
259 Compression (1,)
37500 MakerNote ('e=0.60, tr=25',)
262 PhotometricInterpretation (2,)
270 ImageDescription ('',)
271 Make ('Seek Thermal, Inc.',)
272 Model ('CompactPRO FF',)
273 StripOffsets (646, 33926, 67206, 100486, 133766, 167046, 200326, 233606, 266886, 300166)
338 ExtraSamples (1,)
339 SampleFormat (1,)
277 SamplesPerPixel (4,)
278 RowsPerStrip (26,)
279 StripByteCounts (33280, 33280, 33280, 33280, 33280, 33280, 33280, 33280, 33280, 7680)
282 XResolution ((72, 1),)
283 YResolution ((72, 1),)
296 ResolutionUnit (2,)
50735 CameraSerialNumber ('0E19A0Z16D19',)
305 Software ('Seek Thermal 2.3.0 SDK 1.9.8.11 HW HW: v0 TH: v5 TLID: -1',)
306 DateTime ('2024:03:12 11:47:29',)
316 HostComputer ('SM-A217F, Android 12',)
318 WhitePoint ((3127, 10000), (3290, 10000))
319 PrimaryChromaticities ((640, 1000), (330, 1000), (300, 1000), (600, 1000), (150, 1000), (60, 1000))


In [9]:
tiff_img.getbands()

('R', 'G', 'B', 'A')

In [ ]:
norm_tiff_img  = min_max_norm(tiff_img)

In [ ]:
norm_tiff_img

In [ ]:
plt.imshow(img1[...,:3])

In [ ]:
fig, axs = plt.subplots(1,3)
for i in range(3):
    axs[i].imshow(img2[...,i])
    axs[i].axis('off')

fig, axs = plt.subplots(1,4)
for i in range(4):
    axs[i].imshow(img1[...,i])
    axs[i].axis('off')

min max temperature of each image

In [ ]:
with open('../config.yaml','r') as f:
    config = yaml.safe_load(f)
    
img_paths = sorted(glob.glob(config['dataset']['images']+'/img_thermal_*'))
    

In [ ]:
temp_annot_df = pd.read_csv(list(glob.glob('../../../dataset/*.csv'))[0]).drop('Unnamed: 0', axis=1)
temp_annot_df.image_name = temp_annot_df.image_name.apply(lambda x: x.rstrip('.jpg'))
temp_annot_df.set_index(keys='image_name', inplace=True, drop=True)
temp_annot_df['image_path'] = img_paths

def create_temp_to_color_mapping(image_path):
    img = utils.rotate_to_vertical(image_path)
    spectrum, c_img = utils.extract_image_and_spectrum(img)
    idx = os.path.basename(image_path).rstrip('.jpg')
    min, max = temp_annot_df.loc[idx][['low_temperature', 'high_temperature']]
    print(idx, min, max)

read config and images

In [ ]:
class ArrayKey:
    def __init__(self, array):
        self.array = array

    def __hash__(self):
        return hash(tuple(self.array))

    def __eq__(self, other):
        return np.array_equal(self.array, other.array)

In [ ]:
# plt.imshow(spectrum)
obj = temp_annot_df.iloc[0]
color_bar, c_img = utils.extract_image_and_spectrum(utils.rotate_to_vertical(obj['image_path']))
# color_bar_normalized = color_bar / 255.0 if color_bar.max() > 1 else color_bar
# norm_img = c_img / 255.0 if c_img.max() > 1 else c_img
unique_colors = np.unique(color_bar[:,26], axis=0)
temperatures = np.linspace(obj['high_temperature'], obj['low_temperature'], num=len(color_bar[:,0]))
lut = {tuple(color): temp for color, temp in zip(unique_colors, temperatures)}
# lut = {}

In [ ]:
unique_lens = [(i, len(np.unique(color_bar[:, i], axis=0))) for i in range(color_bar.shape[1])]

In [ ]:
plt.imshow(c_img)

In [ ]:
np.unique(c_img.reshape(-1, 3), axis=0)[1]

In [ ]:
color_bar[5,:]

In [ ]:
plt.imshow(color_bar[5,:])

In [ ]:
# len(lut)

unique_lens

In [ ]:
lut

In [ ]:
temp_img = np.zeros_like(c_img)[...,0]
for x in range(temp_img.shape[0]):
    for y in range(temp_img.shape[1]):
        temp_img[x,y] = lut[tuple(c_img[x,y])]

In [ ]:
np.array(list(lut.keys())[1])

In [ ]:
plt.imshow(color_bar_normalized)

In [ ]:
# list(lut.keys())[2].array
# list(lut.keys())[0] == ArrayKey(color_bar_normalized[-1,0])
# plt.imshow(color_bar_normalized[-1,0][None, None, :])


In [ ]:
plt.imshow(img)

In [ ]:
img = utils.rotate_to_vertical(img_paths[3])
spectrum, cropped_image = utils.extract_image_and_spectrum(image=img)

normalize_image = lambda x: (x - x.mean(axis=(0, 1), keepdims=True)) / x.std(axis=(0, 1), keepdims=True)

plot channels and normalized image

In [ ]:
n_img = normalize_image(x=cropped_image)

fig, axs = plt.subplots(2,4)

axs[0][0].imshow(cropped_image)
axs[0][0].axis('off')

for i in range(1,4):
    axs[0][i].imshow(cropped_image[...,i-1])
    axs[0][i].axis('off')

axs[1][0].imshow(n_img)
axs[1][0].axis('off')
for i in range(1,4):
    axs[1][i].imshow(n_img[...,i-1])
    axs[1][i].axis('off')

# plt.imshow(cropped_image)
plt.show()

Thresholding

In [ ]:
D = 2
ret, thresh1 = cv2.threshold(cropped_image[...,0], 255 // D, 255, cv2.THRESH_BINARY)
ret, thresh2 = cv2.threshold(cropped_image[...,1], 255 // D, 255, cv2.THRESH_BINARY)
ret, thresh3 = cv2.threshold(cropped_image[...,2], 255 // D, 255, cv2.THRESH_BINARY)
fig, (ax1, ax2, ax3, ax4) = plt.subplots(1,4, figsize=(4*2,1*3))

for ax in (ax1, ax2, ax3, ax4):
    ax.axis('off')

ax1.imshow(thresh1)
ax2.imshow(thresh2)
ax3.imshow(thresh3)
ax4.imshow(cropped_image)
plt.show()
# plt.imshow(thresh)

### Different color maps

In [ ]:
spectrum, c_img = utils.extract_image_and_spectrum(utils.rotate_to_vertical(img_paths[0]))

fig, (ax1, ax2, ax3, ax4) = plt.subplots(1,4)

for ax in (ax1, ax2, ax3, ax4):
    ax.axis('off')

cmap_img = cv2.applyColorMap(c_img, colormap=cv2.COLORMAP_RAINBOW)
n_img = normalize_image(cmap_img)
ax1.imshow(cmap_img[...,0])
ax2.imshow(cmap_img[...,1])
ax3.imshow(cmap_img[...,2])
ax4.imshow(cmap_img)
plt.show()

### Grabcut test

In [ ]:
grayscale_img = cv2.cvtColor(cmap_img, cv2.COLOR_RGB2GRAY)
plt.imshow(grayscale_img)
plt.axis('off')
# max_idx = int(contour_areas.iloc[contour_areas.area.argmax()]['index'])

In [ ]:
color_convs = {
    'HLS':cv2.COLOR_RGB2HLS, 
    'HSV': cv2.COLOR_RGB2HSV, 
    'LAB':cv2.COLOR_RGB2LAB, 
    'LUV': cv2.COLOR_RGB2LUV, 
    'YUV': cv2.COLOR_RGB2YUV
}

### Contours from image (histogram equalization, CLAHE)

In [ ]:
def get_simple_contours(image_path, apply_ce=False, apply_blur=False, apply_clahe=False):
    _, c_img = utils.extract_image_and_spectrum(utils.rotate_to_vertical(image_path))
    og_img = c_img
    c_img = cv2.cvtColor(c_img, cv2.COLOR_RGB2GRAY)
    
    if apply_clahe:
        clahe = cv2.createCLAHE(clipLimit=2.0, tileGridSize=(32,32))
        c_img = clahe.apply(c_img)
    
    if apply_ce:
        # Convert image to grayscale and apply histogram equalization
        c_img = cv2.equalizeHist(c_img)
        
    if apply_blur:
        c_img = cv2.GaussianBlur(c_img, (51, 51), 15)
    
    _, thresholded_image = cv2.threshold(c_img, 200, 255, cv2.THRESH_BINARY)
    contours, _ = cv2.findContours(thresholded_image.astype(np.uint8), cv2.RETR_TREE, cv2.CHAIN_APPROX_SIMPLE)
    new_contours = [np.squeeze(c, axis=1) for c in contours]

    contour_areas = pd.DataFrame([(int(i), cv2.contourArea(c)) for i, c in enumerate(contours)])
    contour_areas.columns = ['index','area']
    contour_areas.sort_values('area',ascending=False, inplace=True)
    contour_areas['index'] = contour_areas['index'].astype(int)
    
    return contour_areas, new_contours, c_img, og_img, thresholded_image

In [ ]:
contour_areas, new_contours, c_img, og_img, thresholded_image = get_simple_contours(image_path=img_paths[0], apply_ce=False, apply_clahe=False)

In [ ]:
fig, (ax1, ax2, ax3) = plt.subplots(1,3)
for i, a in enumerate((ax1, ax2, ax3)):
    a.imshow(cmap_img[..., i])
    a.axis('off')


In [ ]:
_, axs = plt.subplots(1,3)
for i in range(3):
    # axs[i].imshow(og_img[...,i])
    axs[i].imshow(cmap_img[...,i])
    axs[i].axis('off')
plt.title('ORIGINAL')
    
for color, color_conv in color_convs.items():
    # cvt_img = cv2.cvtColor(og_img, color_conv)
    cvt_img = cv2.cvtColor(cmap_img, color_conv)
    _, axs = plt.subplots(1,3)
    for i in range(3):
        axs[i].imshow(cvt_img[...,i])
        axs[i].axis('off')
    plt.title(color)

In [ ]:
cvt_img = cv2.cvtColor(cmap_img, cv2.COLOR_RGB2YUV)
# cvt_img = cv2.cvtColor(og_img, cv2.COLOR_RGB2YUV)
fig, (ax1, ax2) = plt.subplots(1,2)
for a in (ax1, ax2):
    a.axis('off')
    
ax1.imshow(cmap_img)
if cvt_img.shape[-1] in (1,3):
    ax2.imshow(cvt_img)

fig, (ax1, ax2, ax3) = plt.subplots(1,3)
for a in (ax1, ax2, ax3):
    a.axis('off')
    
ax1.imshow(cmap_img[...,0])
ax2.imshow(cmap_img[...,1])
ax3.imshow(cmap_img[...,2])

fig, axs = plt.subplots(1,cvt_img.shape[-1])
for i in range(cvt_img.shape[-1]):
    axs[i].axis('off')
    axs[i].imshow(cvt_img[...,i])
    
plt.show()

In [ ]:
def sigmoid(x):
    return 1 / (1 + np.exp(-x))

def np_norm(x):
    return (x - x.mean()) / x.std()

In [ ]:
_, (ax1, ax2) = plt.subplots(1,2)
enhanced_img = cv2.convertScaleAbs(cmap_img[...,1], alpha=2,beta=-40)
ax1.imshow(enhanced_img)
ax1.axis('off')
ax2.imshow(cmap_img[...,1])
ax2.axis('off')
plt.show()

In [ ]:
thresh, binary_image = cv2.threshold(enhanced_img, 200, 255, cv2.THRESH_BINARY)
plt.imshow(binary_image)

In [ ]:
# EROSION/DILATION
kernel = cv2.getStructuringElement(cv2.MORPH_ELLIPSE, (5, 5))  # Adjust kernel size as needed
img_dilate = cv2.dilate(enhanced_img, kernel, iterations=2)
plt.imshow(img_dilate)
plt.show()
thresh, dilated_binary_image = cv2.threshold(img_dilate, 190, 255, cv2.THRESH_BINARY)
plt.imshow(dilated_binary_image)

In [ ]:
# OPENING OPENCV

kernel = cv2.getStructuringElement(cv2.MORPH_ELLIPSE, (51, 51))  # Adjust kernel size as needed
opened = cv2.morphologyEx(dilated_binary_image, cv2.MORPH_OPEN, kernel)

plt.imshow(opened)

In [ ]:
contours, _ = cv2.findContours(dilated_binary_image.astype(np.uint8), cv2.RETR_TREE, cv2.CHAIN_APPROX_SIMPLE)
new_contours = [np.squeeze(c, axis=1) for c in contours]

contour_areas = pd.DataFrame([(int(i), cv2.contourArea(c)) for i, c in enumerate(contours)])
contour_areas.columns = ['index', 'area']
contour_areas.sort_values('area', ascending=False, inplace=True)
contour_areas['index'] = contour_areas['index'].astype(int)
empty_img = np.zeros_like(dilated_binary_image)
plt.imshow(dilated_binary_image)

colors = ['green','blue','red','pink','yellow']

for idx, row in enumerate(contour_areas.iloc[:2].values):
    # rles = M.frPyObjects(new_contours[int(row[0])], h=opened.shape[0], w=opened.shape[1])
    # rle = M.merge(rles)
    # mask = M.decode(rle)
    bbox = cv2.boundingRect(new_contours[int(row[0])])
    plt.gca().add_patch(patches.Rectangle((bbox[0]-10, bbox[1]-10), bbox[2]+20, bbox[3]+20, facecolor='none', edgecolor='red'))
    # plt.gca().add_patch(patches.Polygon(new_contours[int(row[0])], closed=True, facecolor=colors[idx], edgecolor='white'))
plt.show()

### K-Means clustering

In [ ]:
from sklearn.cluster import KMeans

km = KMeans(n_clusters=2, random_state=0)
km.fit(enhanced_img.reshape(-1, 1))

In [ ]:
plt.imshow(km.labels_.reshape(*enhanced_img.shape))

### Contours from edges

In [ ]:
def get_edge_contours(image_path, apply_ce=False, apply_blur=False, apply_clahe=False):
    _, c_img = utils.extract_image_and_spectrum(utils.rotate_to_vertical(image_path))
    og_img = c_img
    if apply_ce:
        # Convert image to grayscale and apply histogram equalization
        c_img = cv2.equalizeHist(cv2.cvtColor(c_img, cv2.COLOR_RGB2GRAY))
    else:
        # Ensure the image is in grayscale if CE isn't applied
        c_img = cv2.cvtColor(c_img, cv2.COLOR_RGB2GRAY)
    
    if apply_clahe:
        clahe = cv2.createCLAHE(clipLimit=2.0, tileGridSize=(8,8))
        c_img = clahe.apply(c_img)

    
    if apply_blur:
        c_img = cv2.GaussianBlur(c_img, (51, 51), 15)
    
    # Apply Canny edge detection
    edges = cv2.Canny(c_img, 50, 50)  # Adjust thresholds as needed

    # Find contours based on edges
    contours, _ = cv2.findContours(edges.astype(np.uint8), cv2.RETR_TREE, cv2.CHAIN_APPROX_SIMPLE)
    new_contours = [np.squeeze(c, axis=1) for c in contours]

    contour_areas = pd.DataFrame([(int(i), cv2.contourArea(c)) for i, c in enumerate(contours)])
    contour_areas.columns = ['index', 'area']
    contour_areas.sort_values('area', ascending=False, inplace=True)
    contour_areas['index'] = contour_areas['index'].astype(int)
    
    return contour_areas, new_contours, c_img, edges


In [ ]:
contour_areas, new_contours, c_img, edges = get_edge_contours(image_path=img_paths[0], apply_ce=False, apply_clahe=True)

_, (ax1, ax2) = plt.subplots(1,2)
ax1.imshow(c_img)
ax2.imshow(edges)
plt.show()

In [ ]:
# max_idx = max(enumerate(contours), key=lambda x: cv2.contourArea(x[1]))[0]
empty_img = np.zeros_like(thresholded_image)
plt.imshow(empty_img)

colors = ['green','blue','red','pink','yellow']

for idx, row in enumerate(contour_areas.iloc[:1].values):
    plt.gca().add_patch(patches.Polygon(new_contours[int(row[0])], closed=True, facecolor=colors[idx], edgecolor='white'))
plt.show()